In [ ]:
from ultralytics import YOLO
import torch
import matplotlib.pyplot as plt
import os
import pandas as pd

# ==========================
# VERIFICAR DISPONIBILIDAD DE GPU
# ==========================
if torch.cuda.is_available():
    dispositivo = 0
    print(" CUDA está disponible. Usando GPU.")
else:
    dispositivo = 'cpu'
    print(" CUDA no está disponible. Usando CPU.")

# ==========================
# ENTRENAMIENTO
# ==========================

# Cargar modelo base preentrenado (YOLOv11-seg)
modelo = YOLO('/workspace/yolo11x-seg.pt')

# Entrenar el modelo
resultados = modelo.train(
    data='/workspace/data.yaml',
    epochs=300,
    imgsz=640,
    batch=16,
    patience=50,  # early stopping
    optimizer='AdamW',  # alternativo: 'SGD'
    lr0=1e-3,  # tasa de aprendizaje inicial
    lrf=1e-5,  # tasa de aprendizaje final (cosine decay)
    warmup_epochs=3,
    weight_decay=0.001,  # regularización
    dropout=0.1,  # evita overfitting
    cos_lr=True,  # learning rate scheduler
    hsv_h=0.015,  # augmentaciones tipo Roboflow
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=0.1,
    translate=0.1,
    scale=0.5,
    shear=0.1,
    perspective=0.0005,
    flipud=0.0,
    fliplr=0.5,
    mosaic=1.0,
    mixup=0.1,
    save=True,
    save_period=10,
    project='runs/',
    name='yolo11x-seg_vegetation',
    exist_ok=True,
    device='0'  # usa 'cpu' si no tienes GPU, '0' para GPU
)

# ==========================
# GRAFICAR ENTRENAMIENTO
# ==========================

ruta_resultados = 'runs/train/yolo11x_seg_vegetation/results.csv'

if os.path.exists(ruta_resultados):
    df = pd.read_csv(ruta_resultados)

    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    plt.plot(df['epoch'], df['train/box_loss'], label='Box Loss')
    plt.plot(df['epoch'], df['train/seg_loss'], label='Seg Loss')
    plt.title('Pérdida durante el entrenamiento')
    plt.xlabel('Época')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(df['epoch'], df['metrics/mAP_0.5'], label='mAP@0.5')
    plt.plot(df['epoch'], df['metrics/mAP_0.5:0.95'], label='mAP@0.5:0.95')
    plt.title('Precisión del modelo')
    plt.xlabel('Época')
    plt.ylabel('mAP')
    plt.legend()

    plt.tight_layout()
    plt.savefig('grafica_entrenamiento.png')
    plt.show()

else:
    print(" No se encontró el archivo de resultados. Asegúrate de que el entrenamiento ha comenzado.")

